# Introduction to post processing and logging with the Empowered Herding simulation
This notebook is intended to provide a high-level overview of the logging from the Empowered Herding simulation.  It's not exhausive documentation(!).

Some terms used in this document are:
- **User / Participant** - the person undertaking the experiment
- **Session** - a single instance of the experiment.  A session starts when the program is first loaded and ends when the program closes.  A session contains multiple trials.
- **Trial** - a single episode of the herding "game".


Original author: Chris Bennett (christopher.bennett@bristol.ac.uk)

Last update: 14/06/2022

## Files

The simulation logs are stored in a direction named with the session id which generated them.  The format of the session id is "yyyymmddThhmmss" e.g. "20220604T151812".  This is the date and time (in ISO 8601 format) when the data was collected.

The following .pkl files should be present in the directory:

 - **user_details**: stores the participant's responses to questions about themselves e.g. whether english is their first language, date of birth etc.
 - **post_test_responses**: stores the participant's responses to the questions asked after each trial e.g. "the sliders".
 - **config_fam_X_simlog**: the simulation data (sim_data) from the familiration trials.  
     - X is numbered 1-4
 - **config_exp_X_Y_Z**: the simuulation data (sim_data) from the experimental trials.  
     - X is numbered 1-12
     - Y is either 
         - absent  (the simulation did not show the state of the dogs empowerement to the participant during the trial)
         - "empshown" (empowerment information was shown)
     - Z is either
         - absent (empowerment was calculated using the "vanilla" method where all states in which the dog changes the state of the flock are counted)
         - "taskweighted" (empowerment was calculated using a method which ones uses states in which the dog effected a change in the flock which moved the flock closer to the goal)

## Data structures

This notebook gives examples for loading the raw data logged by the empowered herding experiment and turning it into (slightly) more user friendly formats.

After running the cells below, the following view of the data should be available:

 - **user_details_df**: a pandas dataframe storing the participant's answers to screening and consent questions
 - **user_responses_df**: a panads dataframe storing the participant's answer to each post-test questionaire (i.e. the sliders). 
     - *user_responses_df.attrs* is a dictionary with two keys: 
         - 'session_id' (the unique identifier for session) 
         - 'test_order' (the names of the config files in the order they were presented to the participant)
 - **events_df** : a pandas data frame which stores everything the user did during the simulation
 - **sim_data['sheep_logs']** : a dictionary referenced by the agent's id which stores when the sheep agent was created, removed and its positions in a structure of type type SimLog.AgentState
 - **sim_data['dog_logs']** : a dictionary referenced by the agent's id which stores when the dog agent was created, removed and its positions in a structure of type type SimLog.AgentState
 
See the section ["Simulation Log Files"](#simulation_log_files) for a walk through for how the simulation stores the actions and state of the simulation at each time step.  


         
         
 




# Setting up the Environment

You will need to edit the paths to match where the simulation logs and empowered_herding program are located

Set the session_id to the unique identifier of the experiment you want to analyse

In [191]:
import os
import numpy as np
import pickle
import glob
import pandas as pd
# user_home_dir = os.path.expanduser('~')
current_dir = os.path.abspath('.')
#change this to where the empowered herding model is 
# os.chdir(current_dir + "/..")#'C:\\Documents\\simulations\\Empowerment_Integration\\Python Model')

#change this to the name of the session to be analysed
session_id = "20220809T095540"

#change this to match where the logs are stored
# base_path = "C:\\Documents\\simulations\\Empowerment_Integration\\Python Model\\Empowerment Results"
base_path = current_dir + "/../Empowerment Results"
#base_path = os.path.join(user_home_dir, "OneDrive - University of Bristol\\Empowerment Results")

path = os.path.join(base_path, session_id)

#change this to match the name of a single log file of the form config_XXXXXX_simlog.pkl 
log_file_name = "config_exp_1_simlog.pkl"

In [192]:
# this is needed as a work around to handle simulation logs created as part of the beta testing
import io
class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        renamed_module = module
        if module == "model.SimLog":
            renamed_module = "model.SimLogBetaVersion"

        return super(RenameUnpickler, self).find_class(renamed_module, name)

def renamed_load(file_obj):
    return RenameUnpickler(file_obj).load()

# Retrieving the Participant's Details
These are the details the participant entered in response to the screening questions and consent forms

In [193]:
#Load the participant's responses
with open(os.path.join(path,"user_details.pkl"), "rb") as input_file:
    user_details = pickle.load(input_file)

In [194]:
print("These are the participant's details")
print("---------------------------")
print(f"Session ID was {user_details[0]}")
print(f"User details where {user_details[1]}")

These are the participant's details
---------------------------
Session ID was 20220809T095540
User details where {'participantnumber': '1002', 'english': (('No', 1), 1), 'vision': (('Yes', 0), 0), 'colour': (('No', 1), 1), 'age': '22', 'gender': (('Male', 1), 1), 'games': '0'}


And as a pandas data frame

In [195]:
user_details_simple = user_details[1]
user_details_simple['english'] = user_details_simple['english'][0][0]
user_details_simple['colour'] = user_details_simple['colour'][0][0]
user_details_simple['vision'] = user_details_simple['vision'][0][0]
user_details_simple['gender'] = user_details_simple['gender'][0][0]
user_details_df =  pd.DataFrame(user_details_simple, index=[0])

with open(os.path.join(path, 'user_details.csv'), 'wb') as output_file:
    user_details_df.to_csv(output_file)

user_details_df

,participantnumber,english,vision,colour,age,gender,games
0,1002,No,Yes,No,22,Male,0


# Retrieving the Participant's Responses to the Post-Trial Questions
These are the responses the participant made with the sliders after each experiment

In [196]:
with open(os.path.join(path,"post_test_responses.pkl"), "rb") as input_file:
    test_responses = pickle.load(input_file)

In [197]:
print(test_responses)

['20220809T095540', ['config_exp_6', 'config_exp_3', 'config_exp_11', 'config_exp_7', 'config_exp_8', 'config_exp_5', 'config_exp_2', 'config_exp_9', 'config_exp_1', 'config_exp_10', 'config_exp_12', 'config_exp_4', 'config_exp_7_empshown', 'config_exp_1_empshown', 'config_exp_9_empshown', 'config_exp_11_empshown', 'config_exp_12_empshown', 'config_exp_5_empshown', 'config_exp_8_empshown', 'config_exp_4_empshown', 'config_exp_6_empshown', 'config_exp_2_empshown', 'config_exp_3_empshown', 'config_exp_10_empshown'], {'config_exp_6': {'time': '2', 'engaged': 1, 'part_of_team': 1}, 'config_exp_3': {'time': '1', 'engaged': 3, 'part_of_team': 3}, 'config_exp_11': {'time': '4', 'engaged': 1, 'part_of_team': 1}, 'config_exp_7': {'time': '5', 'engaged': 3, 'part_of_team': 4}, 'config_exp_8': {'time': '6', 'engaged': 1, 'part_of_team': 5}, 'config_exp_5': {'time': '3', 'engaged': 5, 'part_of_team': 3}, 'config_exp_2': {'time': '8', 'engaged': 5, 'part_of_team': 1}, 'config_exp_9': {'time': '0.4'

and as a pandas dataframe

In [198]:
#convert all the user responses into a pandas dataframe table
user_responses_df = pd.DataFrame.from_dict(test_responses[2], orient='index')
user_responses_df.attrs = {'session_id': test_responses[0], 'test_order' :  test_responses[1]}

with open(os.path.



join(path, 'user_responses.csv'), 'wb') as output_file:
    user_responses_df.to_csv(output_file)

In [199]:
user_responses_df

,time,engaged,part_of_team
config_exp_6,2,1,1
config_exp_3,1,3,3
config_exp_11,4,1,1
config_exp_7,5,3,4
config_exp_8,6,1,5
config_exp_5,3,5,3
config_exp_2,8,5,1
config_exp_9,0.4,5,3
config_exp_1,2.5,6,3
config_exp_10,2,3,5


In [200]:
#Play it safe and sort the data frame in the order the experiments were run (it should already be in this order..)
idx = user_responses_df.index == user_responses_df.attrs['test_order']
user_responses_df = user_responses_df[idx]

In [201]:
print(f"Session ID: {user_responses_df.attrs['session_id']}")
#print(f"test order was {user_responses_df.attrs['test_order']}")
print("in the order tested, responses where")
user_responses_df

Session ID: 20220809T095540
in the order tested, responses where


,time,engaged,part_of_team
config_exp_6,2,1,1
config_exp_3,1,3,3
config_exp_11,4,1,1
config_exp_7,5,3,4
config_exp_8,6,1,5
config_exp_5,3,5,3
config_exp_2,8,5,1
config_exp_9,0.4,5,3
config_exp_1,2.5,6,3
config_exp_10,2,3,5


<a id='simulation_log_files'></a>
# Simulation Log Files

Each trial creates a single file named config_XXXXXX_simlog.pkl where XXXXX describes the parameters used during the trial.  

There (should be!) multiple files with this format inside the directory for a single experiment.

This section follows an example of opening and reading a single log file.

The commands and approach of this section can be extended to open, process and examine multiple files e.g. if we want to compare the performance of different participants.

The first step is to load the file.

Note, the module *"model/SimLog.py"* needs to be in the current directory - *Need to check this and confirm*

In [202]:
os.chdir('..')
try_old_version_b = False
try:
    with open(os.path.join(path, log_file_name), "rb") as input_file:
        sim_data = pickle.load(input_file)
except:
    # the beta sessions created logs using a slightly different version of the data structures
    # (the only difference was userInputs instead of UserInputs!!)
    # Unfortunately, this means we need to explicitly tell pickle to use a different version of the SimLog
    # This work around was taken from https://stackoverflow.com/questions/2121874/python-pickling-after-changing-a-modules-directory
    with open(os.path.join(path, log_file_name), "rb") as input_file:
        sim_data = renamed_load(input_file)

os.chdir('post_processing')        

sim_data is a dictionary which makes uses of two data structures called userInputs* and AgentState to store all the activity which occured during an experiment.
*ok, so bad coding here, this should be capatalised for consistency but 

It has the following five keys:

- **dog_logs** : a dictionary keyed by agent id, stores a log of the state of each dog i.e., 
                  { 1 : AgentState, 2 : AgentState,...dog_id:AgentState,...#Dogs:AgentState}
- **sheep_logs** : a dictionary keyed by agent id, stores a log of the state of each sheep i.e., 
                  { 10000 : AgentState, 10001 : AgentState,...sheep_id:AgentState,...10000+#Sheep:AgentState}
    
- **world_at_t** : a dictionary keyed by simulation step, stores the ids and positions of all the agents present in the world at each simulation step i.e.,
                 t : {  'ids' : 1d numpy array, 
                        'positions' : 2d numpy array where the ith row relates position of the ith id}
                        
- **user_log** :  a data structure which stores all the user interactions which occured during the simulation. It has two parts 
    - *user_log.events_at_t* : the agent events which were caused by user input
    - *user_log.input_at_t* : the user inputs (button presses, times and locations)
                    
                    
- **meta_data**: a dictionary which stores information about the simulation.
    - **config_name**: the name of the config which set the simulation parameters
    - **session_id**: the identifer for the **session** and also the time at which the **session** was started. Note, the session id is **not** the start time of the trial.
    - **start_time**: the time at which the **trial** started
    - **end_time**: the time at which the **trial** ended
    - **taskweighted_empowerment**: True if the empowerement was calculated using a task weighted approach
    - **empowerment_shown**: True if empowerment information (via colour of the dog) was shown to the participant
                    
    

Note, the dog ids start at 1 and the sheep ids start at 10,000.  This is a legacy from the simulations more humble beginnings.  If more than 9,999 dogs are used then the logging will break (although this is practically impossible with a time limit of 2 mins (ish) and a limit of 10 concurrent dogs)
                        
                 
    




In [203]:
print(f"The complete simulation data file for session {session_id}, log file {log_file_name}")
sim_data

The complete simulation data file for session 20220809T095540, log file config_exp_1_simlog.pkl


{'dog_logs': {0: <model.SimLog.AgentState at 0x1188aee90>,
  1: <model.SimLog.AgentState at 0x1198357b0>,
  2: <model.SimLog.AgentState at 0x119836bc0>,
  3: <model.SimLog.AgentState at 0x119837a60>,
  4: <model.SimLog.AgentState at 0x119836aa0>,
  5: <model.SimLog.AgentState at 0x119835a20>,
  6: <model.SimLog.AgentState at 0x1198bd990>,
  7: <model.SimLog.AgentState at 0x1198be1a0>,
  8: <model.SimLog.AgentState at 0x1198bce20>,
  9: <model.SimLog.AgentState at 0x1198bcf10>,
  10: <model.SimLog.AgentState at 0x1198bc400>,
  11: <model.SimLog.AgentState at 0x1198bcca0>},
 'sheep_logs': {99: <model.SimLog.AgentState at 0x1198bcc70>,
  100: <model.SimLog.AgentState at 0x1198bc7c0>},
 'world_at_t': {0: {'ids': array([  0.,   1.,   2.,  99., 100.]),
   'positions': array([[ 80.,  80.],
          [ 80., 140.],
          [140.,  80.],
          [200., 200.],
          [220., 190.]])},
  1: {'ids': array([  0.,   1.,   2.,  99., 100.]),
   'positions': array([[ 80.        ,  78.        ],
  

## Accessing the meta data

In [204]:
print("Meta data for loaded log file")
print("-----------------------------")
sim_data['meta_data']

Meta data for loaded log file
-----------------------------


{'config_name': 'config_exp_1',
 'session_id': '20220809T095540',
 'start_time': datetime.datetime(2022, 8, 9, 10, 9, 1, 587397),
 'end_time': datetime.datetime(2022, 8, 9, 10, 9, 39, 836149),
 'taskweighted_empowerment': False,
 'empowerment_shown': False}

## Converting the positions of each agent at each time step into a big numpy table

It's useful to convert the world data into a big table to make calculating metrics easier

The table is oraganised into the following dimensions:
- **rows (dimesion 0)** are time
- **columns (dimension 1)** are the agent ids, 
- **verticals (dimesion 2)** are the cell connents and these are the agents position

In [205]:
sim_data['dog_logs']

{0: <model.SimLog.AgentState at 0x1188aee90>,
 1: <model.SimLog.AgentState at 0x1198357b0>,
 2: <model.SimLog.AgentState at 0x119836bc0>,
 3: <model.SimLog.AgentState at 0x119837a60>,
 4: <model.SimLog.AgentState at 0x119836aa0>,
 5: <model.SimLog.AgentState at 0x119835a20>,
 6: <model.SimLog.AgentState at 0x1198bd990>,
 7: <model.SimLog.AgentState at 0x1198be1a0>,
 8: <model.SimLog.AgentState at 0x1198bce20>,
 9: <model.SimLog.AgentState at 0x1198bcf10>,
 10: <model.SimLog.AgentState at 0x1198bc400>,
 11: <model.SimLog.AgentState at 0x1198bcca0>}

In [206]:
dog_ids = list(sim_data['dog_logs'].keys())
sheep_ids = list(sim_data['sheep_logs'].keys())
col_names = dog_ids + sheep_ids
row_names =  list(sim_data['world_at_t'].keys())
#create a big table with 3 dimensions (time, agents, xy-position)
# and initialise it with nan
world_ts = np.empty((len(row_names), len(col_names),2))
world_ts[:] = np.nan

#copy the data from the dictionary world_at_t into the big table
for t in sim_data['world_at_t'].keys():
    # time started from 1 and indexing is from 0
    row_idx = t-1
    #copy the ids and positions from the dicitonary just to make it a bit easier to read the code
    ids = sim_data['world_at_t'][t]['ids']
    positions_t = sim_data['world_at_t'][t]['positions']   
    #loop through each id stored at time t, find which column has been allocated to the id 
    # and copy the relevant position from the dictionary to the table
    for i,iid in enumerate(ids):
        col_idx = np.where(col_names == iid)
        #print(f"row idx {row_idx}, col_idx {col_idx}")
        world_ts[row_idx, col_idx,:] = positions_t[i]

In [207]:
#we can now access the positions of agents at time t using a simple numpy reference
t=25
positions_at_t = np.squeeze(world_ts[t-1,:,:])

print(f"At time {t}:")
for i,id in enumerate(col_names):
    print(f"agent {id} was at position {positions_at_t[i,:]}")
print("\n")    
print(f"Note, a value of [nan, nan] means the agent was not present in the world at time {t}")

At time 25:
agent 0 was at position [114.23916308  74.97341791]
agent 1 was at position [124.53684715 139.23356046]
agent 2 was at position [103.17771031  49.37507028]
agent 3 was at position [nan nan]
agent 4 was at position [nan nan]
agent 5 was at position [nan nan]
agent 6 was at position [nan nan]
agent 7 was at position [nan nan]
agent 8 was at position [nan nan]
agent 9 was at position [nan nan]
agent 10 was at position [nan nan]
agent 11 was at position [nan nan]
agent 99 was at position [199.99414002 200.00337399]
agent 100 was at position [220.20552576 186.00661827]


Note, a value of [nan, nan] means the agent was not present in the world at time 25


## Converting the positions of each agent at each time step into a pandas dataframe
**This isn't working because pandas initialises its cells to NaN (size (1,1)) and we then want to overwrite the cells one by one with a position of size (2,1).**

The numpy code is almost certainly faster but the pandas approach might be easier to read if it worked!

In [208]:
# import pandas as pd
# import numpy as np
# dog_ids = list(sim_data['dog_logs'].keys())
# sheep_ids = list(sim_data['sheep_logs'].keys())
# # col_names = []
# # for iid in dog_ids+sheep_ids:
# #     col_names.append(str(iid) +'x')
# #     col_names.append(str(iid) +'y')
# # print(col_names)

# col_names = dog_ids + sheep_ids
# #create the empty data frame with the columns are the agent ids and the row index is the time step
# world_data_df = pd.DataFrame(index = sim_data['world_at_t'].keys(), columns = col_names, dtype='object')

In [209]:
#world_data_df

In [210]:
# #now need to populate the cells.  There's probably a fast way to do this but for simplicity, just loop through all the world_at_t entries
# for t in sim_data['world_at_t'].keys():
#     row_idx = world_data_df.index==t
#     ids = sim_data['world_at_t'][t]['ids']
#     positions_t = sim_data['world_at_t'][t]['positions']
#     for i,iid in enumerate(ids):
#         col_idx = world_data_df.columns == iid
#         world_data_df.loc[row_idx, col_idx] = [positions_t[i,:]]

## The 'dog_logs' and 'sheep_logs' structures
The logger contains the state information for each agent as a simLog.AgentState class

To access the log of a single agent's state:

* **for dogs**: sim_data["dog_logs"][*agent_number*]
* **for sheep**: sim_data["sheep_logs"][*agent_number*]

Note, the states are only recorded for the time when the agent was present in the simulation.  I.e. if a dog is removed then the state record for that dog ends.  Each new dog is added with a new id and starts a new state record.

In [211]:
sim_data['dog_logs'][1]

The following properties are available:

'id' : an integer

'state' : a dictionary with the following entries

        {
        id': integer
        'time_created' : simulation tick #
        'time_destroyed': simulation tick #
        'postions' : np 2d array where dimension 0 is time and dimesion 1 is x,y position
        'empowerment' : np 1d array where cell n is the empowerment of the agent at time ['time'][n]
        'time' : the simulation ticks when the agent state was updated, the ith cell relates to the ith cell for 'position' and 'empowerment'
        }
 
 For example, for a dog agent the state information can be accessed as follows:

In [212]:
idog = 1
id = sim_data['dog_logs'][idog].id
tc = sim_data['dog_logs'][idog].state['time_created']
td = sim_data['dog_logs'][idog].state['time_destroyed']
p = sim_data['dog_logs'][idog].state['positions']
time = sim_data['dog_logs'][idog].state['time']
empowerment = sim_data['dog_logs'][idog].state['empowerment']

print(f"Dog with Id {id}")
print(f'at simulation step {tc} the dog was created')
n=4
print(f'at simulation step {time[n]} the dog was at position {p[n,:]}')
print(f'at simulation step {time[n]} the dog had empowerment {empowerment[n]}')
print(f'at simulation step {td} the dog was removed (-1 indicates it was present at the end of the simulation)')

Dog with Id 1
at simulation step 0 the dog was created
at simulation step 3 the dog was at position [ 80.71482967 134.07170678]
at simulation step 3 the dog had empowerment 5
at simulation step -1 the dog was removed (-1 indicates it was present at the end of the simulation)


And to see a single agents complete state type:
sim_data['*type*_logs'][*agent_number*].state

e.g., to see the state of the sheep with id=10000

In [213]:
try:
    print(sim_data['sheep_logs'][10000].state)
except:
    #the beta tests started the sheep ids from 100 so if it's an older log then try this instead
    print(sim_data['sheep_logs'][100].state)

{'id': 100, 'time_created': 0, 'time_destroyed': -1, 'positions': array([[220.        , 190.        ],
       [220.        , 190.        ],
       [220.00039983, 189.99980008],
       ...,
       [601.77549723, 453.18242834],
       [603.76287923, 453.4067334 ],
       [605.71793947, 453.82832499]]), 'time': array([   0,    0,    1, ..., 1562, 1563, 1564]), 'empowerment': array([-1, -1, -1, ..., -1, -1, -1])}


To plot the empowerment of a dog over the course of a run, combine together the logged empowerment value and the time at which the value was logged. 

In [214]:
#e.g. for dog with id 1.  The first column is simulation tick and the second column is empowerment
for i in zip(sim_data['dog_logs'][1].state['time'], sim_data['dog_logs'][1].state['empowerment']):
    print(i)

(0, -1)
(0, 0)
(1, 5)
(2, 5)
(3, 5)
(4, 5)
(5, 5)
(6, 5)
(7, 5)
(8, 5)
(9, 5)
(10, 5)
(11, 5)
(12, 5)
(13, 5)
(14, 5)
(15, 5)
(16, 5)
(17, 5)
(18, 5)
(19, 5)
(20, 5)
(21, 5)
(22, 5)
(23, 5)
(24, 5)
(25, 5)
(26, 5)
(27, 5)
(28, 5)
(29, 5)
(30, 5)
(31, 5)
(32, 5)
(33, 5)
(34, 5)
(35, 5)
(36, 5)
(37, 5)
(38, 5)
(39, 5)
(40, 6)
(41, 6)
(42, 6)
(43, 8)
(44, 8)
(45, 9)
(46, 10)
(47, 10)
(48, 11)
(49, 12)
(50, 12)
(51, 13)
(52, 13)
(53, 13)
(54, 13)
(55, 13)
(56, 13)
(57, 13)
(58, 13)
(59, 13)
(60, 13)
(61, 13)
(62, 13)
(63, 13)
(64, 13)
(65, 13)
(66, 13)
(67, 13)
(68, 12)
(69, 12)
(70, 12)
(71, 12)
(72, 11)
(73, 11)
(74, 11)
(75, 11)
(76, 11)
(77, 11)
(78, 10)
(79, 10)
(80, 10)
(81, 10)
(82, 9)
(83, 9)
(84, 10)
(85, 9)
(86, 9)
(87, 8)
(88, 8)
(89, 8)
(90, 7)
(91, 8)
(92, 8)
(93, 7)
(94, 8)
(95, 7)
(96, 8)
(97, 7)
(98, 7)
(99, 8)
(100, 8)
(101, 7)
(102, 8)
(103, 7)
(104, 8)
(105, 7)
(106, 8)
(107, 7)
(108, 8)
(109, 7)
(110, 8)
(111, 7)
(112, 8)
(113, 8)
(114, 7)
(115, 8)
(116, 8)
(117, 8)
(11

Using numpy, we can combine the empowerment time series for all dogs into a big table and call it empowerment_ts.  

In [215]:
import numpy as np
dog_ids = list(sim_data['dog_logs'].keys())
col_names = dog_ids
row_names =  list(sim_data['world_at_t'].keys())
#create a big table with 2 dimensions (time, agents)
# and initialise it with nan
empowerment_ts = np.empty((len(row_names), len(col_names)))
empowerment_ts[:] = np.nan

#copy the data from the dictionary world_at_t into the big table
for dog_id in sim_data['dog_logs'].keys():
    dog_state = sim_data['dog_logs'][dog_id].state
    #copy the empowerment
    col_idx = np.where(np.asarray(col_names) == dog_id)
    row_idx = dog_state['time']
    empowerment_ts[row_idx, col_idx] = dog_state['empowerment']

print("The full empowerment table is...")
print("(A value of nan means the agent was not present at that time step)")
print("\n")
empowerment_ts


The full empowerment table is...
(A value of nan means the agent was not present at that time step)




array([[ 0.,  0.,  0., ..., -1., -1., -1.],
       [ 0.,  5.,  5., ..., nan, nan, nan],
       [ 0.,  5.,  5., ..., nan, nan, nan],
       ...,
       [nan,  8., nan, ...,  8.,  0.,  5.],
       [nan,  8., nan, ...,  8.,  0.,  5.],
       [nan,  4., nan, ...,  9.,  0., 10.]])

In [216]:
print("Or if you really want to handle it as a pandas dataframe")
col_names = ["dog"+str(i) for i in range(empowerment_ts.shape[1])]
df = pd.DataFrame(empowerment_ts, columns = col_names)
df.index.name = 'tick'
df

Or if you really want to handle it as a pandas dataframe


,dog0,dog1,dog2,dog3,dog4,dog5,dog6,dog7,dog8,dog9,dog10,dog11
tick,,,,,,,,,,,,
0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1560,NaN,8.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,8.0,0.0,3.0
1561,NaN,8.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,8.0,0.0,4.0
1562,NaN,8.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,8.0,0.0,5.0


Dimesion 0 is time and dimension 1 is the dog id.  The [i,j] cell is therefore the empowerement of dog j at simulation step i.

In [217]:
t = 3
dog_id = 1
# python indexes from 0 but ids start at 1
dog_id = dog_id - 1
print("the dimensions of the table are:" + " " + str(empowerment_ts.shape))
print(f"the empowerment of dog {dog_id} at t={t} was {empowerment_ts[t,dog_id]}")
print(f"Note, a value of nan means the agent was not present in the world at time {t}")


the dimensions of the table are: (1565, 12)
the empowerment of dog 0 at t=3 was 0.0
Note, a value of nan means the agent was not present in the world at time 3


## Accessing the logs of what the participant did
sim_data['user_log'] is a data structure which stores all the user interactions which occured during the simulation. 

It has two parts
- **sim_data['user_log'].events_at_t** : the agent events which were caused by user input
- **sim_data['user_log'].input_at_t** : the user inputs (button presses, times and locations)

### sim_data['user_log'].events_at_t 
When a user interacts with the simulation, a data "frame" is created in the form of a dictionary with the following structure:

    {
        'id' : ordered list of the agent id's affected by the user
        'event : ordered list of the action taken on each id (either add or remove)
        'grid_position' : the grid square the mouse click occured in (this is **not** the same as the screen coordinates)
    }

The frames are stored in a further dictiory that can be indexed by the simulation timestep at which the event took place

e.g. *sim_data['user_log'].events_at_t[t]* accesses the "frame" at time t.

and *sim_data['user_log'].events_at_t[5]['id']* will return the list of affected ids at t=5.

To see **all** time steps the particpant interacted on

In [218]:
sim_data['user_log'].events_at_t

{0: {'id': [0, 1, 2, 99, 100, 3, 4, 5, 6, 7, 8, 9, 10, 11],
  'event': ['add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add',
   'add'],
  'position': [array([80, 80]),
   array([ 80, 140]),
   array([140,  80]),
   array([200, 200]),
   array([220, 190]),
   array([298, 168]),
   array([ 10, 212]),
   array([388, 272]),
   array([276, 298]),
   array([419, 320]),
   array([506, 366]),
   array([437, 443]),
   array([574, 460]),
   array([578, 322])]},
 <function matplotlib.pyplot.tick_params(axis='both', **kwargs)>: {'id': [-6,
   -2,
   -5,
   -3,
   0,
   -8],
  'event': ['destroy', 'destroy', 'destroy', 'destroy', 'destroy', 'destroy'],
  'position': [array([380.41297842, 326.13222896]),
   array([368.8247323 , 324.01764716]),
   array([340.62889274, 281.858302  ]),
   array([262.62637509, 263.69098474]),
   array([136.0717353 , 382.13870656]),
   array([163.66386831, 387.12066459])]}}

and to see what the user did at time t...

In [219]:
t=10
if t not in sim_data['user_log'].events_at_t.keys():
    print(f"no participant interaction at t={t}")
else:
    print(sim_data['user_log'].events_at_t[t])

no participant interaction at t=10


### sim_data['user_log'].input_at_t
The actual inputs the participant made are recorded and stored in sim_data['user_log'].input_at_t.

The indexing and structure is very similar to sim_data['user_log'].events_at_t except the frame created when a user interacts with the simulation has different fields:

    {
        'type': the name of the (mouse) button pressed
        'screen_position': the position of the mouse pointer on the **screen** when the button was pressed
        'grid_position': the position of the mouse pointer on the **window** when the button was pressed
        'realtime': the realworld time of the event, has the format datetime.datetime(yyyy, mm, dd, hh, mm, ss, microsecond)

    }
    
**Important note**, because in principle more than one event can occur at a single time step, each of the fields is a python list.

So to see **all** the inputs recorded during the simulation...

In [220]:
sim_data['user_log'].input_at_t

{56: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(298, 168)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 2, 959523)]},
 177: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(10, 212)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 5, 931953)]},
 358: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(388, 272)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 10, 359544)]},
 401: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(276, 298)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 11, 415624)]},
 460: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(419, 320)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 12, 845882)]},
 513: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(506, 366)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 14, 133673)]},
 573: {'type': ['MB:DOWN:LEFT'],
  'screen_position': [(437, 443)],
  'realtime': [datetime.datetime(2022, 8, 9, 10, 9, 15, 584369)]},
 708: {'type': ['MB:DOWN:RIGHT'],
  'screen_position': [(37

We can see the specific input the user made on time step t by typing sim_data['user_log'].input_at_t[t]

In [221]:
t=12
if t not in sim_data['user_log'].input_at_t.keys():
    print(f"no participant interaction at t={t}")
else:
    print(sim_data['user_log'].input_at_t[t])

no participant interaction at t=12


and to access the time of the input at timestep t

In [222]:
t=12
if t not in sim_data['user_log'].input_at_t.keys():
    print(f"no participant interaction at t={t}")
else:
    print(sim_data['user_log'].input_at_t[t]['realtime'][0])

no participant interaction at t=12


### Create a pandas data frame of the user events
Instead of accessing events using the dictonary stuctures of the previous section, we can combine events_at_t and input_at_t into a single pandas data frame holding all the events for a single trial.  

The approach can be extended to create a list of dataframes (or your favourite flavour of array-like structure) to hold the results from multiple trials and experiments. 



First load the log file...

In [223]:
# we did this at the start of the simulation section

Next, load the "input_at_t" dictionary into a data frame indexed by the times at which events took place.  

In [224]:
df_user_inputs = pd.DataFrame.from_dict(sim_data['user_log'].input_at_t, orient='index')
df_user_inputs.index.name = 'sim_time'

In [225]:
df_user_inputs

,type,screen_position,realtime
sim_time,,,
56,[MB:DOWN:LEFT],"[(298, 168)]",[2022-08-09 10:09:02.959523]
177,[MB:DOWN:LEFT],"[(10, 212)]",[2022-08-09 10:09:05.931953]
358,[MB:DOWN:LEFT],"[(388, 272)]",[2022-08-09 10:09:10.359544]
401,[MB:DOWN:LEFT],"[(276, 298)]",[2022-08-09 10:09:11.415624]
460,[MB:DOWN:LEFT],"[(419, 320)]",[2022-08-09 10:09:12.845882]
513,[MB:DOWN:LEFT],"[(506, 366)]",[2022-08-09 10:09:14.133673]
573,[MB:DOWN:LEFT],"[(437, 443)]",[2022-08-09 10:09:15.584369]
708,[MB:DOWN:RIGHT],"[(373, 312)]",[2022-08-09 10:09:18.856500]
721,[MB:DOWN:RIGHT],"[(382, 337)]",[2022-08-09 10:09:19.170771]


Now load the events_at_t into a data frame indexed by the times at which events took place.

In [226]:
events_df = pd.DataFrame.from_dict(sim_data['user_log'].events_at_t, orient='index')
events_df.index.name = 'sim_time'

In [227]:
events_df

,id,event,position
sim_time,,,
0,"[0, 1, 2, 99, 100, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[add, add, add, add, add, add, add, add, add, ...","[[80, 80], [80, 140], [140, 80], [200, 200], [..."
<function tick_params at 0x11647c4c0>,"[-6, -2, -5, -3, 0, -8]","[destroy, destroy, destroy, destroy, destroy, ...","[[380.41297842420494, 326.1322289567222], [368..."


Finally, combine user_inputs and events_df into a single data frame called events_df

In [228]:
#do a bit of processing to concatinate the user inputs and the agent events into a single table
# TODO doesn't work with continouous space
df = events_df
df['screen_click_coords'] = df_user_inputs['screen_position']
df['user_input'] = df_user_inputs['type']
df['grid_click_coords'] = df_user_inputs['grid_position']
df['time_of_input'] = df_user_inputs['realtime']
df.rename(columns = {'id':'agent_id', 'grid_position' : 'event_position'})
events_df = df

KeyError: 'grid_position'

In [ ]:
events_df

,id,event,position,screen_click_coords,user_input
sim_time,,,,,
0,"[0, 1, 2, 99, 100]","[add, add, add, add, add]","[[80, 80], [80, 140], [140, 80], [200, 200], [...",NaN,NaN
<function tick_params at 0x7f45e90d0ee0>,[0],[destroy],"[[304.02829597676396, 280.26833938124315]]",NaN,NaN


Side note, there are two measurement systems for position used in the simulation.

The simulation uses a grid world to track the position of the agents.  grid_position is where the participant clicked on that grid.  

The simulation displays the graphical elements at a given resolution.  screen_click_coords is the where the participant clicked on the screen relative to the screen's top left position (TODO: check it's definietly top left!).

